In [2]:
import pandas as pd
import os

os.chdir('C:/Documents/COURS/M2/NLP')
# Import data
df = pd.read_csv('apec.csv')

In [3]:
df.head()

,title,type_job,salary,compagny,location,description,source
0,Data scientist,1 CDD de 11 mois,A négocier,Levio,Luxembourg,La division « Database Administration & Data M...,apec
1,Data scientist,1 CDI,40 - 80 k€ brut annuel,LUX-ADVISORY SARL,Luxembourg,La division « Database Administration & Data M...,apec
2,Data scientist,1 CDI,30 - 56 k€ brut annuel,Kaisens data,Fontenay-sous-Bois - 94,"En tant que Data Scientist, vos missions sont ...",apec
3,Data scientist,Meudon - 92,A négocier,1 CDI,NaN,Si pour vous faire grandir les relations humai...,apec
4,Data scientist,1 CDI,A partir de 50 k€ brut annuel,STORIGIN,Meudon - 92,Descriptif de mission :En tant que Data Analys...,apec


In [15]:
from SQLite import data_base


In [13]:
for i in range(len(df['salary'].unique())):
    item = (i+1, df['salary'].unique()[i])
    print(item)
    print('next')

(1, 'A négocier')
next
(2, '40 - 80 k€ brut annuel')
next
(3, '30 - 56 k€ brut annuel')
next
(4, 'A partir de 50 k€ brut annuel')
next
(5, 'A partir de 30 k€ brut annuel')
next
(6, '30 - 45 k€ brut annuel')
next
(7, 'A partir de 40 k€ brut annuel')
next
(8, '45 - 55 k€ brut annuel')
next
(9, 'A partir de 35 k€ brut annuel')
next
(10, '26 - 27 k€ brut annuel')
next
(11, '36 - 45 k€ brut annuel')
next
(12, '34 - 40 k€ brut annuel')
next


In [12]:
df

,title,type_job,salary,compagny,location,description,source
0,Data scientist,1 CDD de 11 mois,A négocier,Levio,Luxembourg,La division « Database Administration & Data M...,apec
1,Data scientist,1 CDI,40 - 80 k€ brut annuel,LUX-ADVISORY SARL,Luxembourg,La division « Database Administration & Data M...,apec
2,Data scientist,1 CDI,30 - 56 k€ brut annuel,Kaisens data,Fontenay-sous-Bois - 94,"En tant que Data Scientist, vos missions sont ...",apec
3,Data scientist,Meudon - 92,A négocier,1 CDI,NaN,Si pour vous faire grandir les relations humai...,apec
4,Data scientist,1 CDI,A partir de 50 k€ brut annuel,STORIGIN,Meudon - 92,Descriptif de mission :En tant que Data Analys...,apec
5,Data scientist,1 CDI,A partir de 30 k€ brut annuel,ABG Consulting,Paris 01 - 75,"Au sein du pôle data science, vous intégrerez ...",apec
6,Data scientist,1 CDI,30 - 45 k€ brut annuel,Amiltone,Lyon 09 - 69,Vos missions ?Intégré à nos équipes sur nos pr...,apec
7,Data scientist,Montreuil - 93,A négocier,1 CDI,NaN,Dans le cadre de la réalisation de sa mission ...,apec
8,Data scientist,1 CDI,A partir de 40 k€ brut annuel,DEVOTEAM,Levallois-Perret - 92,Issu(e) d’une formation supérieure Bac +5 ou p...,apec
9,Data scientist,Meudon - 92,45 - 55 k€ brut annuel,1 CDI,NaN,Si pour vous faire grandir les relations humai...,apec


In [23]:
bd = data_base()

for i in range(len(df['salary'].unique())):
    item = (i+1, df['salary'].unique()[i])
    bd.insert_salaire(item)


bd.commit_change()
bd.close_connection()

AttributeError: 'data_base' object has no attribute 'close_connection'